## Titanic Dataset

In [13]:
import datalabframework as dlf

In [14]:
dlf.project.rootpath()

'/home/jovyan/src'

## Data

In [15]:
metadata = dlf.params.metadata()
metadata

{'engines': {'numpy': {'context': 'numpy'},
  'pandas': {'context': 'pandas'},
  'spark': {'config': {'jobname': 'test', 'master': 'local[4]'},
   'context': 'spark'}},
 'loggers': {'kafka': {'enable': False,
   'hosts': ['localhost:29092'],
   'severity': 'info',
   'topic': 'dlf'},
  'stream': {'enable': True, 'severity': 'info'}},
 'providers': {'local': {'rootpath': '../data', 'service': 'fs'}},
 'resources': {'.etl.clean.test': {'format': 'parquet',
   'path': 'datasets/clean/test',
   'provider': 'local'},
  '.etl.clean.train': {'format': 'parquet',
   'path': 'datasets/clean/train',
   'provider': 'local'},
  '.etl.extract.test': {'format': 'parquet',
   'path': 'datasets/extract/test',
   'provider': 'local'},
  '.etl.extract.train': {'format': 'parquet',
   'path': 'datasets/extract/train',
   'provider': 'local'},
  '.etl.features.test': {'format': 'parquet',
   'path': 'datasets/features/test',
   'provider': 'local'},
  '.etl.features.train': {'format': 'parquet',
   'path'

#### Init Spark

In [16]:
engine = dlf.engines.get('spark')
spark = engine.context()

In [17]:
#print out name and version
'{}:{}'.format(engine.info['context'], spark.sparkSession.version)

'spark:2.3.0'

## Train

#### Load Dataset

In [18]:
df = engine.read('.etl.raw.train', header=True, inferSchema=True)
for column in df.schema:
    print('{:<12} {:<10} {}'.format(column.name, str(column.dataType)[:-4], column.nullable))

PassengerId  Integer    True
Survived     Integer    True
Pclass       Integer    True
Name         String     True
Sex          String     True
Age          Double     True
SibSp        Integer    True
Parch        Integer    True
Ticket       String     True
Fare         Double     True
Cabin        String     True
Embarked     String     True


#### Save as parquet

In [19]:
engine.write(df, '.etl.extract.train', mode='overwrite')

#### Explore the dataset

Check for Null or NaN values, and count them per column

In [20]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



## Test

#### Load Dataset

In [21]:
df = engine.read('.etl.raw.test', header=True, inferSchema=True)
for column in df.schema:
    print('{:<12} {:<10} {}'.format(column.name, str(column.dataType)[:-4], column.nullable))

PassengerId  Integer    True
Pclass       Integer    True
Name         String     True
Sex          String     True
Age          Double     True
SibSp        Integer    True
Parch        Integer    True
Ticket       String     True
Fare         Double     True
Cabin        String     True
Embarked     String     True


#### Save as parquet

In [22]:
engine.write(df,'.etl.extract.test', mode='overwrite')

#### Explore the dataset

In [23]:
for column in df.schema:
    print('{:<12} {:<10} {}'.format(column.name, str(column.dataType)[:-4], column.nullable))

PassengerId  Integer    True
Pclass       Integer    True
Name         String     True
Sex          String     True
Age          Double     True
SibSp        Integer    True
Parch        Integer    True
Ticket       String     True
Fare         Double     True
Cabin        String     True
Embarked     String     True


Check for Null or NaN values, and count them per column

In [24]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|     0|   0|  0| 86|    0|    0|     0|   1|  327|       0|
+-----------+------+----+---+---+-----+-----+------+----+-----+--------+

